# Training a model
see README.md for details

Loosely following this tutorial:
https://medium.com/nlplanet/a-full-guide-to-finetuning-t5-for-text2text-and-building-a-demo-with-streamlit-c72009631887

In [1]:
import os

os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
from tqdm import tqdm
from glob import glob

import torch
import wandb
from datasets import Dataset, Features, load_dataset
from evaluate import load
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer
from transformers import T5Config, T5Model
from transformers import AutoTokenizer
from transformers import ByT5Tokenizer  # a "dummy" tokenizer, tokenizing into bytes
from transformers import DataCollatorWithPadding
from transformers import DataCollatorForSeq2Seq

from config import data_root, model_root, checkpoint_name
from config import token_len, annot_len

In [2]:
torch.cuda.is_available()

False

In [3]:
dfs = []
for fname in glob(f"{data_root}/*.csv"):
    dfs += [pd.read_csv(fname, names=["inputs", "labels"])]
df = pd.concat(dfs, axis=0)

ds = Dataset.from_pandas(df)
ds = ds.train_test_split(test_size=0.1)
ds

DatasetDict({
    train: Dataset({
        features: ['inputs', 'labels', '__index_level_0__'],
        num_rows: 36993
    })
    test: Dataset({
        features: ['inputs', 'labels', '__index_level_0__'],
        num_rows: 4111
    })
})

In [4]:
# tokenizer = ByT5Tokenizer()
tokenizer = AutoTokenizer.from_pretrained("google/byt5-small")


# Create a tokenization function
def preprocess_function(examples):
    inputs = examples["inputs"]
    targets = examples["labels"]
    model_inputs = tokenizer(inputs, max_length=64, truncation=True)

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=64, truncation=True)

    model_inputs["decoder_input_ids"] = labels["input_ids"]
    return model_inputs


# Apply the tokenization function to the dataset
tds = ds.map(preprocess_function, batched=True)
# tds = tds.remove_columns(["input", "label"])
tds

Map:   0%|          | 0/36993 [00:00<?, ? examples/s]

Map:   0%|          | 0/4111 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['inputs', 'labels', '__index_level_0__', 'input_ids', 'attention_mask', 'decoder_input_ids'],
        num_rows: 36993
    })
    test: Dataset({
        features: ['inputs', 'labels', '__index_level_0__', 'input_ids', 'attention_mask', 'decoder_input_ids'],
        num_rows: 4111
    })
})

In [5]:
exact_match_metric = load("exact_match")

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_root}/byT5-ocs-ue",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    # torch_empty_cache_steps=100,
    disable_tqdm=False,
    report_to=None,  # disable wandb.ai
)

config = T5Config.from_pretrained("t5-base")
# config.task_specific_params = {}
model = T5Model(config)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [6]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tds["train"],
    eval_dataset=tds["test"],
    # train_dataset=small_train_dataset,
    # eval_dataset=small_eval_dataset,
    #     data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=exact_match_metric,
)

In [7]:
trainer.train()


The following columns in the training set  don't have a corresponding argument in `T5Model.forward` and have been ignored: labels, __index_level_0__, inputs.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 36993
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 4
  Total optimization steps = 27744
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if y

KeyError: 'loss'

In [ ]:
tds